In [1]:
# --- Celda de Prueba Directa para buscar_coches_bq ---
import pandas as pd
import logging
import traceback
import json # Para imprimir dicts de forma legible

# Ajusta las rutas de importación según tu estructura
from utils.bigquery_tools import buscar_coches_bq 
from config.settings import ( 
    MIN_MAX_RANGES,PENALTY_PUERTAS_BAJAS,PENALTY_LOW_COST_POR_COMODIDAD,
    PENALTY_DEPORTIVIDAD_POR_COMODIDAD,UMBRAL_LOW_COST_PENALIZABLE_SCALED, # Renombrado para claridad
    UMBRAL_DEPORTIVIDAD_PENALIZABLE_SCALED, # Renombrado
    PENALTY_ANTIGUEDAD_MAS_10_ANOS, PENALTY_ANTIGUEDAD_7_A_10_ANOS,
    PENALTY_ANTIGUEDAD_5_A_7_ANOS, BONUS_DISTINTIVO_ECO_CERO_C,
    PENALTY_DISTINTIVO_NA_B, BONUS_OCASION_POR_IMPACTO_AMBIENTAL,
    BONUS_ZBE_DISTINTIVO_FAVORABLE,PENALTY_ZBE_DISTINTIVO_DESFAVORABLE
)

In [3]:

# Configurar el logging para ver los mensajes de depuración de la función
logging.basicConfig(level=logging.INFO) 
# logging.getLogger("utils.bigquery_tools").setLevel(logging.DEBUG) # Descomenta para logs DEBUG de la función

# --- Configuración de Pandas para Mejor Visualización ---
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None) # Mostrar todas las columnas
pd.set_option('display.width', 2000)    
pd.set_option('display.max_colwidth', 100) 

# --- 1. Escenario de Prueba: "Familiar Práctico con Énfasis en Impacto Ambiental" ---
print("\n--- ESCENARIO 1: FAMILIAR PRÁCTICO - IMPACTO AMBIENTAL ---")
filtros_test_familiar_eco = {
    # Filtros técnicos directos
    "estetica_min": 1.0,
    "premium_min": 1.0,
    "singular_min": 2.0,
    "tipo_mecanica": ['HEVG', 'HEVD', 'PHEVG', 'PHEVD', 'BEV', 'GLP'], # Favoreciendo eco
    "tipo_carroceria": ['SUV', 'MONOVOLUMEN', 'FAMILIAR'],
    "plazas_min": 5, # Familia
    "transmision_preferida": "automático", # Para comodidad familiar

    # Filtro económico (Modo 2 Contado)
    "modo": 2,
    "submodo": 1,
    "pago_contado": 30000.0,
    
    # Flags (simulando lo que calcularía finalizar_y_presentar_node)
    "penalizar_puertas_bajas": True, # Asumimos niños y uso frecuente
    "flag_penalizar_low_cost_comodidad": False, # Asumimos comodidad no es 10/10 para activar esto
    "flag_penalizar_deportividad_comodidad": False,
    "flag_penalizar_antiguo_por_tecnologia": True, # Si tecnología es importante y queremos coches más nuevos
    "aplicar_logica_distintivo_ambiental": True, # Si rating_impacto_ambiental fue >= 8
    "es_municipio_zbe": True # Simular que está en ZBE
}

# Pesos (ya normalizados, simulando la salida de normalize_weights)
# Estos deben sumar 1.0. Aquí un ejemplo donde impacto ambiental y sus derivados son altos.
pesos_test_familiar_eco = { 
    "estetica": 0.01, "premium": 0.01, "singular": 0.01, 
    "altura_libre_suelo": 0.03, "batalla": 0.03, "indice_altura_interior": 0.04,
    "ancho_general_score": 0.05, "traccion": 0.02, "reductoras": 0.0,
    "rating_fiabilidad_durabilidad": 0.07, "rating_seguridad": 0.08, 
    "rating_comodidad": 0.06,
    "rating_impacto_ambiental": 0.10, # Peso directo del rating
    "rating_tecnologia_conectividad": 0.05,
    "devaluacion": 0.04,
    "maletero_minimo_score": 0.05, "maletero_maximo_score": 0.05,
    "largo_vehiculo_score": 0.03,
    "autonomia_vehiculo": 0.05,
    "fav_bajo_peso": 0.06, # Activado por alto rating_impacto_ambiental
    "fav_bajo_consumo": 0.08, # Activado por alto rating_impacto_ambiental
    "fav_bajo_coste_uso_directo": 0.0, # Omitido por ahora
    "fav_bajo_coste_mantenimiento_directo": 0.0, # Omitido
    "par_motor_remolque_score": 0.0, "cap_remolque_cf_score": 0.0, "cap_remolque_sf_score": 0.0,
    "fav_menor_superficie_planta": 0.01, "fav_menor_diametro_giro": 0.01,
    "fav_menor_largo_garage": 0.01, "fav_menor_ancho_garage": 0.01, "fav_menor_alto_garage": 0.01,
    "deportividad_style_score": 0.0, "fav_menor_rel_peso_potencia_score": 0.0,
    "potencia_maxima_style_score": 0.0, "par_motor_style_score": 0.0, 
    "fav_menor_aceleracion_score": 0.0
}
# Asegurar que la suma de pesos sea aproximadamente 1.0 (ajusta los valores de arriba)
# print(f"Suma de pesos para test_familiar_eco: {sum(pesos_test_familiar_eco.values())}")

k_test = 5

print(f"\n--- Probando buscar_coches_bq con Escenario 1 ---")
print(f"Filtros para BQ: {json.dumps(filtros_test_familiar_eco, indent=2)}")
# print(f"Pesos para BQ: {json.dumps(pesos_test_familiar_eco, indent=2)}") # Descomentar para ver pesos

resultados_bq_directo = []
sql_ejecutada_directo = ""
params_ejecutados_directo = []

try:
    resultados_tupla = buscar_coches_bq(
        filtros=filtros_test_familiar_eco, 
        pesos=pesos_test_familiar_eco, 
        k=k_test
    )
    if isinstance(resultados_tupla, tuple) and len(resultados_tupla) == 3:
        resultados_bq_directo, sql_ejecutada_directo, params_ejecutados_directo = resultados_tupla
        print(f"\n--- SQL Ejecutada (Escenario 1) ---")
        print(sql_ejecutada_directo)
        # print(f"\n--- Parámetros SQL (Escenario 1) ---")
        # print(json.dumps(params_ejecutados_directo, indent=2))
    else:
        print("WARN: buscar_coches_bq no devolvió la tupla esperada.")
        resultados_bq_directo = resultados_tupla if isinstance(resultados_tupla, list) else []

except Exception as e:
    print(f"ERROR al ejecutar buscar_coches_bq directamente: {e}")
    traceback.print_exc()

print(f"\n--- Resultados Búsqueda Directa BQ ({len(resultados_bq_directo)} coches) ---")
if resultados_bq_directo:
    df_resultados_directo = pd.DataFrame(resultados_bq_directo)
    
    # Lista de columnas que te interesa ver (incluyendo las _scaled y el score)
    columnas_a_ver = [
        'nombre', 'marca', 'modelo', 'precio_compra_contado', 'score_total',
        'tipo_mecanica', 'tipo_carroceria', 'plazas', 'puertas', 
        'distintivo_ambiental', 'ocasion', 'anos_vehiculo',
        'fiabilidad', 'durabilidad', 'seguridad', 'comodidad', 'tecnologia', 
        'peso_original_kg', 'consumo_original', 
        # Algunas _scaled clave para este perfil
        'fiabilidad_scaled', 'durabilidad_scaled', 'seguridad_scaled', 'comodidad_scaled',
        'bajo_peso_scaled', 'bajo_consumo_scaled' 
    ]
    columnas_existentes = [col for col in columnas_a_ver if col in df_resultados_directo.columns]
    
    if 'score_total' in df_resultados_directo.columns: # Formatear score para mejor lectura
         df_resultados_directo['score_total'] = df_resultados_directo['score_total'].apply(lambda x: f"{x:.4f}" if isinstance(x, float) else x)
            
    if columnas_existentes:
        display(df_resultados_directo[columnas_existentes])
    else:
        print("No se encontraron columnas esperadas para mostrar. Mostrando DataFrame completo:")
        display(df_resultados_directo)
else:
    print("No se encontraron resultados para este escenario.")

# --- Puedes añadir más escenarios de prueba aquí ---
# print("\n--- ESCENARIO 2: ... ---")
# filtros_test_2 = { ... }
# pesos_test_2 = { ... }
# ... y repetir la llamada y visualización ...



--- ESCENARIO 1: FAMILIAR PRÁCTICO - IMPACTO AMBIENTAL ---

--- Probando buscar_coches_bq con Escenario 1 ---
Filtros para BQ: {
  "estetica_min": 1.0,
  "premium_min": 1.0,
  "singular_min": 2.0,
  "tipo_mecanica": [
    "HEVG",
    "HEVD",
    "PHEVG",
    "PHEVD",
    "BEV",
    "GLP"
  ],
  "tipo_carroceria": [
    "SUV",
    "MONOVOLUMEN",
    "FAMILIAR"
  ],
  "plazas_min": 5,
  "transmision_preferida": "autom\u00e1tico",
  "modo": 2,
  "submodo": 1,
  "pago_contado": 30000.0,
  "penalizar_puertas_bajas": true,
  "flag_penalizar_low_cost_comodidad": false,
  "flag_penalizar_deportividad_comodidad": false,
  "flag_penalizar_antiguo_por_tecnologia": true,
  "aplicar_logica_distintivo_ambiental": true,
  "es_municipio_zbe": true
}
--- 🧠 SQL Query (buscar_coches_bq) ---

    WITH ScaledData AS (
        SELECT
            *,
            COALESCE(SAFE_DIVIDE(COALESCE(estetica, 1.0) - 1.0, NULLIF(10.0 - 1.0, 0)), 0) AS estetica_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(premium

INFO:root:✅ BigQuery query ejecutada, 5 resultados obtenidos.



--- SQL Ejecutada (Escenario 1) ---

    WITH ScaledData AS (
        SELECT
            *,
            COALESCE(SAFE_DIVIDE(COALESCE(estetica, 1.0) - 1.0, NULLIF(10.0 - 1.0, 0)), 0) AS estetica_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(premium, 1.0) - 1.0, NULLIF(10.0 - 1.0, 0)), 0) AS premium_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(singular, 1.0) - 1.0, NULLIF(10.0 - 1.0, 0)), 0) AS singular_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(altura_libre_suelo, 79.0) - 79.0, NULLIF(314.0 - 79.0, 0)), 0) AS altura_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(batalla, 1650.0) - 1650.0, NULLIF(4035.0 - 1650.0, 0)), 0) AS batalla_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(indice_altura_interior, 0.9) - 0.9, NULLIF(2.7 - 0.9, 0)), 0) AS indice_altura_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(ancho, 1410.0) - 1410.0, NULLIF(2164.0 - 1410.0, 0)), 0) AS ancho_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(fiabilidad, 1.0) - 1.0, NULLIF(10.0 - 1.0, 0))

,nombre,modelo,precio_compra_contado,score_total,tipo_mecanica,tipo_carroceria,plazas,distintivo_ambiental,ocasion,anos_vehiculo,seguridad,comodidad,tecnologia,peso_original_kg,consumo_original,fiabilidad_scaled,durabilidad_scaled,seguridad_scaled,comodidad_scaled,bajo_peso_scaled,bajo_consumo_scaled
0,Honda CR-V 2.0 i-MMD e:HEV Lifestyle 2WD e-CVT,CR-V,27400,1.0697,HEVG,SUV,5,ECO,True,5,8.6,10.0,7.0,1614.0,33.6,0.888889,1.000000,0.844444,1.0,0.622442,0.791401
1,Honda CR-V 2.0 i-MMD e:HEV Lifestyle 2WD e-CVT,CR-V,28690,1.0697,HEVG,SUV,5,ECO,True,5,8.6,10.0,7.0,1614.0,33.6,0.888889,1.000000,0.844444,1.0,0.622442,0.791401
2,BMW X1 xDrive25e Steptronic (DKG),X1,21250,1.0684,PHEVG,SUV,5,0,True,3,9.2,10.0,9.0,1930.0,18.4,0.777778,0.888889,0.911111,1.0,0.518152,0.912420
3,BMW X1 xDrive25e Steptronic (DKG),X1,26990,1.0684,PHEVG,SUV,5,0,True,3,9.2,10.0,9.0,1930.0,18.4,0.777778,0.888889,0.911111,1.0,0.518152,0.912420
4,BMW X1 xDrive25e Steptronic (DKG),X1,26990,1.0684,PHEVG,SUV,5,0,True,3,9.2,10.0,9.0,1930.0,18.4,0.777778,0.888889,0.911111,1.0,0.518152,0.912420
